# 🧠 Chapter 15: Graph Deep Learning on Point Clouds

Point clouds are irregular structures. Standard CNNs (grids) don't apply. We can either voxelize (lossy) or use **Graph Neural Networks (GNNs)** or **PointCNN** to learn directly on the points structure.

**Content:**
1.  **PointCNN Concept**: Learning an X-Transformation to canonicalize point order.
2.  **Implementation**: Building a simple PointCNN layer.
3.  **Training**: Training on Indoor LiDAR data.

In [ ]:
import numpy as np
import laspy
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.neighbors import KDTree
import time

## 1. Load Data

We use indoor LAS datasets (`train` and `test`).

In [ ]:
train_path = "../DATA/indoor_train.las"
test_path = "../DATA/indoor_test.las"

def load_las(path):
    try:
        las = laspy.read(path)
        points = np.vstack((las.x, las.y, las.z)).transpose()
        try:
            labels = las.classification
        except:
            labels = np.zeros(len(points), dtype=int)
        return points, labels
    except FileNotFoundError:
        print(f"File {path} not found. Creating dummy data.")
        return np.random.rand(1000, 3), np.random.randint(0, 5, 1000)

train_points, train_labels = load_las(train_path)
print(f"Train points: {train_points.shape}")

## 2. PointCNN Core: X-Conv

The core idea is to learn a transformation $X$ that weights and permutes input points into a latent canonical form, allowing convolution.

For this educational notebook, we will illustrate the **K-Nearest Neighbors** selection which is the first step of the convolution.

In [ ]:
# Finding K-Nearest Neighbors for Representative Points

def get_neighbors(points, k=16):
    # Sample representative points (e.g., Farthest Point Sampling or Random)
    # Here, simple random sampling for speed
    num_rep_points = 1024
    indices = np.random.choice(len(points), num_rep_points, replace=False)
    rep_points = points[indices]
    
    tree = KDTree(points)
    dist, ind = tree.query(rep_points, k=k)
    
    return rep_points, points[ind] # [N_rep, 3], [N_rep, K, 3]

start = time.time()
rep_pts, neighbors = get_neighbors(train_points)
print(f"Neighborhood search took {time.time() - start:.4f}s")
print(f"Neighbors shape: {neighbors.shape}")

## 3. Defining a Simplified PointNet/Graph Layer

Instead of full PointCNN (which is complex code), we implement a simplified **GraphConv** logic using PyTorch: Feature aggregation from neighbors.

In [ ]:
class SimpleGraphLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_channels, out_channels),
            nn.ReLU(),
            nn.Linear(out_channels, out_channels)
        )
        
    def forward(self, x, neighbors_indices):
        # x: [N, C] features
        # neighbors_indices: [N, K]
        
        # Gather neighbor features
        # This is high-level pseudo-logic for brevity; standard implementation requires sophisticated indexing
        # In practice we use pytorch_geometric or heavy indexing ops.
        pass

For practical Deep Learning on Point Clouds, libraries like **PyTorch Geometric** or **Open3D-ML** are recommended over writing raw layers from scratch unless for research.